<a href="https://colab.research.google.com/github/ravishankarramakrishnan/Deep_Learning_Practice/blob/master/Translation_Machine_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Common Libraries

import numpy as np
import pandas as pd
import math
import re
import time
from google.colab import drive

In [0]:
# Importing the Deep Learning Libraries

try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds # For Tokenizing the Sentences

In [0]:
# Mount the Drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Importing the Data

In [0]:
# Loading all the Datasets

# Loading the English Dataset

with open("/content/drive/My Drive/Mordern NLP - Python/Translator using Transformers/europarl-v7.fr-en.en", 
          mode='r',
          encoding='utf-8') as f:
      europarl_en = f.read()

# Loading the French Dataset

with open("/content/drive/My Drive/Mordern NLP - Python/Translator using Transformers/europarl-v7.fr-en.fr", 
          mode='r',
          encoding='utf-8') as f:
      europarl_fr = f.read()

# Loading the Non Breaking English Dataset

with open("/content/drive/My Drive/Mordern NLP - Python/Translator using Transformers/nonbreaking_prefix.en", 
          mode='r',
          encoding='utf-8') as f:
      non_breaking_prefix_en = f.read()

# Loading the Non Breaking French Dataset

with open("/content/drive/My Drive/Mordern NLP - Python/Translator using Transformers/nonbreaking_prefix.fr", 
          mode='r',
          encoding='utf-8') as f:
      non_breaking_prefix_fr = f.read()

In [0]:
# Check for the Dataset

print(europarl_en[:50]) # Get first 50 Chars
print(europarl_fr[:50]) # Get first 50 Chars
print(non_breaking_prefix_en[:5]) # Get first 50 Chars
print(non_breaking_prefix_fr[5:10]) # Get first 50 Chars

Resumption of the session
I declare resumed the se
Reprise de la session
Je déclare reprise la sessio
a
b
c

d
e



## Cleaning the Data

In [0]:
# Get non breaking prefixes as clean list with space and dot at beginning and end as it can ease the Data Cleaning Process

# For English
# Split the Values by \n
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
# Add Space adn Dot at beginning and End
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]

# For French
# Split the Values by \n
non_breaking_prefix_fr = non_breaking_prefix_fr.split("\n")
# Add Space adn Dot at beginning and End
non_breaking_prefix_fr = [' ' + pref + '.' for pref in non_breaking_prefix_fr]

In [0]:
# Validation

non_breaking_prefix_fr

[' a.',
 ' b.',
 ' c.',
 ' d.',
 ' e.',
 ' f.',
 ' g.',
 ' h.',
 ' i.',
 ' j.',
 ' k.',
 ' l.',
 ' m.',
 ' n.',
 ' o.',
 ' p.',
 ' q.',
 ' r.',
 ' s.',
 ' t.',
 ' u.',
 ' v.',
 ' w.',
 ' x.',
 ' y.',
 ' z.',
 ' mme.',
 ' mlle.',
 ' c.-à-d.',
 ' cf.',
 ' chap.',
 ' e.g.',
 ' al.',
 ' etc.',
 ' ex.',
 ' fig.',
 ' suiv.',
 ' sup.',
 ' suppl.',
 ' tél.',
 ' vol.',
 ' vs.']

In [0]:
# Do for the Whole Corpus

# Tokenize function will do jobs like lowercase, space before, fullstop, comma etc, as the tokenizer function does it
# We want to remove the Points that are not full stops
# So we need each word and symbol that we want to keep in lowercase and seperated by spaces so we can tokenize them. So each point that is'nt a fullstop will have 3 ### sign
# Perform this is non breaking Prefix datafile

# Do for English Corpus
corpus_en = europarl_en

# Replace . with ###
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + "###")
# If we have . followed by any charecters other than spaces, then it isnt considered fullstop and we add ### here
corpus_en = re.sub(r"\.(?=[0-9]|[A-Z]|[a-z])", ".###", corpus_en)
corpus_en = re.sub(r".###", " ", corpus_en)
# Remove Extra Whitespaces
corpus_en = re.sub(r"  +", " ", corpus_en)
corpus_en = corpus_en.split("\n")

# Do for French Corpus
corpus_fr = europarl_fr

# Replace . with ###
for prefix in non_breaking_prefix_fr:
    corpus_fr = corpus_fr.replace(prefix, prefix + "###")
# If we have . followed by any charecters other than spaces, then it isnt considered fullstop and we add ### here
corpus_fr = re.sub(r"\.(?=[0-9]|[A-Z]|[a-z])", ".###", corpus_fr)
corpus_fr = re.sub(r".###", " ", corpus_fr)
# Remove Extra Whitespaces
corpus_fr = re.sub(r"  +", " ", corpus_fr)
corpus_fr = corpus_fr.split("\n")

In [0]:
print(corpus_en[:5])
print(corpus_fr[:5])

['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You have requested a debate on this subject in the course of the next few days, during this part-session.', "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union."]
['Reprise de la session', 'Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacan

In [0]:
# Tokenizing the Sentences
# Tokenization transforms each charecters into one corresponding number

tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_en, target_vocab_size = 2**13)

tokenizer_fr = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_fr, target_vocab_size = 2**13)

# It will take a lot of time to Run

In [0]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2 # 2 - One before and After a word
VOCAB_SIZE_FR = tokenizer_fr.vocab_size + 2 # 2 - One before and After a word
print(VOCAB_SIZE_EN, VOCAB_SIZE_FR)

8186 8168


In [0]:
# Get the Inputs and Outputs from the Datasets - -2 start, -1 end

inputs = [ [VOCAB_SIZE_EN - 2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN - 1] for sentence in corpus_en]
outputs = [ [VOCAB_SIZE_FR - 2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR - 1] for sentence in corpus_fr]

In [0]:
# Removing Long Sentences from the Corpus
# Issues: When we pad all sentences, we want all sentences in same length for the Batch to work, and it'll take a lot of time to train if very long sentences are taken
MAX_LENGTH = 20

# Perform Processing for Inputs
# To test if len of sentence is too high, we can get the Count of the Index in the very sentence
idx_to_remove = [ count for count, sent in enumerate(inputs) if len(sent) > MAX_LENGTH ]

# If we want to remove 2 and 4 th element (for elements), the elements may shift to the previous or the next one, so we process in reverse order so we process and remove the right element

for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

# Perform Processing for Outputs
# To test if len of sentence is too high, we can get the Count of the Index in the very sentence
idx_to_remove = [ count for count, sent in enumerate(outputs) if len(sent) > MAX_LENGTH ]

# If we want to remove 2 and 4 th element (for elements), the elements may shift to the previous or the next one, so we process in reverse order so we process and remove the right element

for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

In [0]:
# Inputs and Outputs Creation

# Pad Inputs and Outputs to bring each sentence to the Same Length
# Padding - Add 0 at the end(post) of each sentence for Inputs
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                            value=0,
                                            padding="post",
                                            maxlen = MAX_LENGTH)

# Padding - Add 0 at the end(post) of each sentence for Outputs
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                            value=0,
                                            padding="post",
                                            maxlen = MAX_LENGTH)

In [0]:
# Creating our Datasets

BATCH_SIZE = 64
BUFFER_SIZE = 20000 # Shuffling of the Datasets

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

# To improve the way dataset is stored and have access, increasing the speed of training etc, we use
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# To Improve sped of training and have access to data faster 
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## Model Building

![Transformer Architecture](https://i0.wp.com/blog.exxactcorp.com/wp-content/uploads/2019/05/1_blSbN23mOGMZ_DWvTAcO1w.png)

In [0]:
# First Step: Creating the Embeddings and the Positional Encoding. We use Position Embeddings sin and cos as defined in the paper of the Transformer

# Positional Encoding Layer

class PositionalEncoding(layers.Layer):

    def __init__(self):
        super(PositionalEncoding, self).__init__()

    # Peform Embedding Based on Formula, sin for even and cos for odd

    def get_angles(self, pos, i, d_model): # Shape should be pos: (seq_length, 1)., i: (1, d_model)
        angles = 1/np.power(10000., (2 ** (i // 2)) / np.float(d_model))
        return pos * angles # To get shape as (seq_length, d_model)

    def call(self, inputs):
        seq_length = inputs.shape.as_list()[-2]
        d_model = inputs.shape.as_list()[-1]

        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis], # Creating seq_length, 1
                                 np.arange(d_model)[np.newaxis, :],    # Creating 1, d_model
                                 d_model)
        
        # For Even
        angles[:, 0::2] = np.sin(angles[:, 0::2]) # Take all columns, Start from Row 0 with Stride of 2
        # For Odd
        angles[:, 1::2] = np.cos(angles[:, 1::2]) # Take all columns, Start from Row 1 with Stride of 2

        pos_encoding = angles[np.new_axis, ...] # Add New Axis and ... means Keep Everything Else
        return inputs + tf.cast(pos_encoding, tf.float_32)

In [0]:
# Create Scaled Dot Product Attention - Attention Mechanism

# Attention Computation: Attention(Q, k, v) = softmax(Q * k.T/ sqrt(d.k)) * v

# Function for Scaled Dot Product

def scaled_dot_product_attention(queries, keys, values, mask): # Mask is Lookahead Mask that doesnt allow decoder to see next word or padding mask that pads 0 at end of each sentence
    product = tf.matmul(queries, keys, transpose_b= True) # transpose_b shows the 2nd matrix (keys) to be transposed
    # Get the Values of dK
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float_32) # Get all dimensions of the keys and take the Last one as it corresponds to Embedding

    # Get Scaled Dot Product
    scaled_product = product / tf.math.sqrt(keys_dim)

    # Add the Mask (May be Optional As per Paper)
    if mask is not None:
        # Multiply by -infinity as we are going to apply Softmax next. Softmax gives values between 0 and 1, and the sum of all equals 1
        scaled_product += (mask * -1e9)  # We take 1e9 as close to infinity, as softmax takes the 0's as very low weightage
    
    # Get the Attention Mechanism Formula
    attention = tf.matmul(tf.nn.softmax(scaled_product, axis = -1), values) # Axis= -1 is to be applied on last axis., softmax(Q * k.T/ sqrt(d.k)) - gives weightage, so we apply softmax to make it to 1

    return attention

In [0]:
# Create Multi Headed Attention SubLayer

# Linear Function is dense layer in tf, split result into sub space and apply scaled dot product

class MultiHeadAttention(layers.Layer): # It inherits from layers.Layer

    def __init(self, nb_proj):
        super(MultiHeadAttention, self).__init__()
        self.nb_proj = nb_proj

    # Define Build, Similar to init, here if we use the Object for the First time, Unlike init (called when object is created), It is called the first time the object is used
    def build(self, input_shape): # Input Shape will exactly be the Shape of Queries
        self.d_model = input_shape[-1] # Last Dimension of the Inputs
        # Dimension of Each Projection = dim of model / nb of projection
        # Make Sure to when dividing as integers, keep it and others will be 0. Assert demands to have this Condition
        assert self.d_model % self.nb_proj == 0

        self.d_proj = self.d_model // self.nb_proj

        self.query_lin = layers.Dense(units= self.d_model)
        self.key_lin = layers.Dense(units= self.d_model)
        self.value_lin = layers.Dense(units= self.d_model)
        self.final_lin = layers.Dense(units= self.d_model)

    # Write Function for split_proj
    def split_proj(self, inputs, batch_size): # Batch size is used to reshape the Inputs so it is splitted into subspaces, Shape will be inputs: (batch_size, seq_length, d_model)
        shape = (batch_size, -1, self.nb_proj, self.d_proj)
        splited_inputs = tf.reshape(inputs, shape = shape) # The Shape will be (batch_size, seq_length, nb_proj, d_proj)
        # We need to Alter the Outputs to get result shape as (batch_size, nb_proj, seq_length, d_proj)
        return tf.transpose(splited_inputs, perm= [0,2,1,3])
    
    # Writing Architecture of the Sub Layer
    def call(self, queries, keys, values, mask):

        batch_size = tf.shape(queries)[0]

        # Apply Huge Linear Function to them
        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)
        # Query, Key, Value.. lin are 3 Dense Layers that we declare now

        # Split the Inputs after applying the Linear Function - To Subspaces
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)

        # Compute the Attention for Each of those SubSpaces
        attention = scaled_dot_product_attention(queries, keys, values, mask)

        # Perform Concatenation - Opposite of Split_proj, precisely the splited_inputs method. We invert the Dimension once again
        attention = tf.transpose(attention, perm= [0,2,1,3])

        concat_attention = tf.reshape(attention, shape= (batch_size, -1, self.d_model))

        outputs  = self.final_lin(concat_attention)

        return outputs

In [0]:
# Build the Encoder

class EncoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
        super(EncoderLayer, self).__init__()

        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout

    def build(self, input_shape):

        self.d_model = input_shape[-1]
        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate = self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon= 1e-6)
        self.dense_1 = layers.Dense(units= self.FFN_units, activation= "relu")
        self.dense_2 = layers.Dense(units= self.d_model, activation= "relu")
        self.dropout_2 = layers.Dropout(rate = self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon= 1e-6)


    # We use 3 inputs here because 3 (Q, K, V) are same, so we use 3 inputs
    def call(self, inputs, mask, training):
        attention = self.multi_head_attention(inputs,
                                              inputs,
                                              inputs,
                                              mask)
        attention = self.dropout_1(attention, training= training)
        attention = self.norm_1(attention + inputs)

        # Create Feed Forward Part
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs)
        outputs = self.norm_2(outputs + attention)

        return outputs

In [0]:
# Create Whole Encoder

class Encoder(layers.Layer):

    def __init__(self, nb_layers, FFN_units, nb_proj, dropout, vocab_size, d_model, name="encoder"):
        super(Encoder, self).__init__(name= name)
        self.nb_layers = nb_layers
        #self.FFN_units = FFN_units
        #self.nb_proj = nb_proj
        #self.dropout = dropout
        self.d_model = d_model
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate = dropout)
        self.enc_layers = [EncoderLayer(FFN_units,
                                        nb_proj,
                                        dropout)
                          for _ in range(nb_layers)]

    def call(self, inputs, mask, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # As defined in Paper
        outputs = self.pos_encoding(outputs) # It returns the Sum of the Outputs and the Positional Encoding
        outputs = self.dropout(output, training)

        # Get Layers as list
        for i in range(self.nb_layers):
            outputs = self.enc_layers[i](outputs, mask, training)
        
        return outputs

In [0]:
# Build the Decoder

# Create the Decoder Sub Layer

class DecoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout): # nb_proj - Used for Multihead Attention Sublayer; FFN_units - Feed Forward Network
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout

    def build(self, input_shape): # Shape of Input
        self.d_model = input_shape[-1] # To get the Last Dimension
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate= self.dropouts)
        self.norm_1 = layers.LayerNormalization(epsilon= 1e-6)

        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate= self.dropouts)
        self.norm_2 = layers.LayerNormalization(epsilon= 1e-6)

        self.dense_1 = layers.Dense(units= self.FFN_units,
                                    activation= "relu")
        self.dense_2 = layers.Dense(units= self.d_model)
        
        self.dropout_3 = layers.Dropout(rate= self.dropout)
        self.norm_3 = layers.LayerNormalization(epsilon= 1e-6)

    # Layer that uses Multihead Inputs after Positional Encoding
    def call(self, inputs, enc_outputs, mask_1, mask_2, training): # 2 Masks for Self Attention Layer and Attention Layer which uses outputs of the Encoder
        attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1) # Q,K,V are same, so we use 3 inputs here
        attention = self.dropout_1(attention, training)
        attention = self.norm_1(attention + inputs)

        # Attention Layer that uses the Output of the Encoder
        attention_2 = self.multi_head_attention_2(attention, enc_outputs, enc_outputs, mask_2) # Q - attention, K,V - Output of Encoder, as defined in the Paper
        attention_2 = self.dropout_2(attention_2, training) # Training is Boolean, whether True or False
        attention_2 = self.norm_2(attention_2 + attention)

        # Encoder Layer, A Simple Feed Forward Sequence

        outputs = self.dense_1(attention_2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention_2)

        return outputs

In [0]:
# Build the Decoder

class Decoder(layers.Layer):

    def __init__(self, nb_layers, FFN_units, nb_proj, dropout, vocab_size, d_model, name="decoder"):
        super(Decoder, self).__init__(name= name)
        self.d_model = d_model
        self.nb_layers = nb_layers
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate= dropout)

        self.dec_layers = [ DecoderLayer(FFN_units,
                                         nb_proj,
                                         dropout) for _ in range(nb_layers) ]

    def call(self, inputs, enc_outputs, mask_1, mask_2, training):

        # Write Code for Decoder to till Multiheaded attention, taken care by decoding layer
        outputs = self.embedding(inputs)
        outputs += tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)

        # Apply Multiple Decoder Sublayers

        for i in range(self.nb_layers):
            outputs = self.decoder_layers[i](outputs,
                                             enc_outputs,
                                             mask_1,
                                             mask_2,
                                             training)

        return outpus

In [0]:
# Building the TransFormer

# Create 2 Mask tokens, First to mask padding tokens at <EOS> and other to create Lookahead Mask, that makes sure decoder has access to Future Masks

class Transformer(tf.keras.Model):

    def __init__(self, vocab_size_enc, vocab_size_dec, d_model, nb_layers, FFN_units, nb_proj, dropout, name= "transformer"): # 2 Embeddings, One for Encoder and other for Decoder
        super(Transformer, self).__init__(name= name)

        self.encoder = Encoder(nb_layers, FFN_units, nb_proj, dropout, vocab_size_enc, d_model)
        self.decoder = Decoder(nb_layers, FFN_units, nb_proj, dropout, vocab_size_dec, d_model)
        self.last_linear = layers.Dense(units= vocab_size_dec) # Size is same as decoder vocab as we want to get the Probablity of Each possible word in our Vocab to be the Next one

    # Build 2 Masking Functions
    def create_padding_mask(self, seq): # Shape of seq is (batch_size, seq_length)
        # Look for 0's in the Padding token
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32) # Represent it as float, Cast - Casts a tensor to a new datatype.
        return mask[:, tf.newaxis, tf.newaxis, :] # This Mask is applied to Attention Computation & Multihead Attention Sublayer, Before Scaled Dot Product Attention. Add 2 New Dimension
        # Before Applying scaled dotproduct attention, we split data via split proj. So that is the Reason for Adding new Axis. Thanks to Empty Dimension (NewAxis), the Row will be repeated as many times as we need

        # Check on for Matrix Multiplication - Create Matrix that cant have access to Future words, So we use Triangular Matrix
        def create_look_ahead_mask(self, seq):
            seq_len = tf.shape(seq)[1] # Get the Shape of Matrix and 1st element
            look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0) # Create a Triangular Matrix and add 0 to top right part of the Matrix. -1 says we dont want to use this one and 0 says we want j to be >= i
            return look_ahead_mask

        # Combine the Encoder and Decoder Architecture

        def call(self, enc_inputs, dec_inputs, training):
            # Add Just Simple Padding Mask for the Encoder
            enc_mask = self.create_padding_mask(enc_inputs)
            # Add Lookahead Mask for the Decoder - see example for Clarity
            dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs),
                                  self.create_look_ahead_mask(dec_inputs))
            # Add 2nd Mask of the Decoder
            dec_mask_2 = self.create_padding_mask(enc_inputs) # This is for 2nd Multihead mask where we apply mask for encoder and decoder inputs
            # This is for Rearrangement of Outputs of Encoder, following the way Sentences from the Decoder are related to sentence from the Decoder
            # The Final Shape of the Multihead Attention Layer will have the Same Shape as what we have inside the Decoder, and way we acheive this is by combining the Information from the Encoder and decoder (Matrix Multiplication)

            enc_outputs = self.encoder(enc_inputs, enc_mask, training)
            dec_outputs = self.decoder(dec_inputs, enc_outputs, dec_mask_1, dec_mask_2, training)

            outputs = self.last_linear(dec_outputs)

            return outputs


## Training the Model

In [0]:
# Clear all the Sessions
tf.keras.backend.clear_session()

# Define the Hyperparameters - After Comments # is what was defined on the Paper of Transformers
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT = 0.1 # 0.1

# Building the Transformer

transformer = Transformer(vocab_size_enc= VOCAB_SIZE_EN,
                          vocab_size_dec = VOCAB_SIZE_FR,
                          d_model = D_MODEL,
                          nb_layers = NB_LAYERS,
                          FFN_units = FFN_UNITS,
                          nb_proj = NB_PROJ,
                          dropout = DROPOUT)

In [0]:
# Build Functions Related to Loss and Accuracy
# We use Categorical and get Probablities as to which word will be the next one (inside our Vocabulary)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True, # OP of Model are real nos that becomes Probablity in future by applying Softmax
                                                            reduction= "none") # We Dont want one Loss number to be there for the Whole Batch

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0)) # Get rid of All the Losses that Corresponds to 0 in Targets
    loss_ = loss_object(target, pred)
    # Multiply Mask to Loss

    # Make Mask Same Type
    mask = tf.cast(mask, dtype= loss_.dtype) # Now Values inside loss and mask will be of Same type
    # Multiply Loss with Mask - for putting 0's on the Padding Tokens So, we only use Projections for Real sentence and not just 0's in the End
    loss_ *= mask

    return tf.reduce_mean(loss_) # return what we get after having sum over all Batches/Dimensions

# Create Train Loss that Keeps track of Losses during training

train_loss = tf.keras.metrics.Mean(name= "train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name= "train_accuracy")

In [0]:
# As given in paper, we use the Optimizer to indicate on how to use the Gradients and Optimize model

# Create Custom Learning Rate

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    # To Create Own Custom Learning Rate

    def __init__(self, d_model, warmup_steps= 4000): # 4000 as defined in the Paper
        super(CustomSchedule, self).__init__()

        self.d_model= tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    # Write Call Function at initialisation, as we dont need to add any more Layers
    def __call__(self, step): # Step is always the Parameter in Learning Rate Scheduler. It'll deal by itself
        # Implement as shown in Paper
        arg1 = tf.math.rsqrt(step) # Reciprocal of Sqrt Elementwise => Power^-0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
# Now Create a Custom Learning Rate

learning_rate = CustomSchedule(D_MODEL)

# Create the Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate,
                                     beta_1= 0.9,
                                     beta_2= 0.98,
                                     epsilon= 1e-9)

In [0]:
# Create CheckPoints and Store the Model at the End of each Epoch

checkpoint_path = "./drive/My Drive/Mordern NLP - Python/Translator using Transformers/ckpt/"

# Create Checkpoint Object - To Store the State of Optimizer and Transformer

ckpt = tf.train.Checkpoint(transformer= transformer,
                           optimizer= optimizer)
# Create Checkpoint Manager
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep= 5) # Store 5 Different Checkpoints

# Check if Checkpoint Already exists

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint Restored")

In [0]:
# Create Loop to Train the Model

EPOCHS = 10

for epoch in range(EPOCHS):
    print("Start of Epoch {}".format(epoch+1))
    start = time.time()

    # Clean History Of Losses and Accuracy at the Beginning of an Epoch
    train_loss.reset_states()
    train_accuracy.reset_states()

    # Iterate over Each Batch
    for (batch, (enc_inputs, targets)) in enumerate(dataset): # Our Dataset Contains input and Output, so we get it as Tuple
        dec_inputs = targets[:, :-1] # Target - last words
        dec_outputs_real = targets[:, 1:] # Get after the First Word
        # Apply the Transformer
        with tf.GradientTape() as tape: # Tape Records everything that happens while prediction
            predictions = transformer(enc_inputs, dec_inputs, True) # True for Training
            loss = loss_function(dec_outputs_real, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        # Add to Train Loss
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)

        # Pint Loss and Accuracy from Time to Time

        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(epoch+1,
                                                                         batch,
                                                                         train_loss.result(),
                                                                         train_accuracy.result()))
        # At End of Epoch, Save the Model to Checkpoint
        ckpt_save_path = ckpt_manager.save()
        print("Saving Checkpoint for Epoch {} at {}".format(epoch+1, ckpt_save_path))

        print("Time Taken for 1 Epoch is: {} secs \n".format(time.time() - start))

## Model Evaluation

In [0]:
# Get the Input from User and Encode the Values

def evaluate(inp_sentence):
    # Add Starting and Ending Token and Pad the Words
    inp_sentence = [VOCAB_SIZE_EN - 2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN - 1] # -2 for Starting Token, -1 for Ending Token
    # Simulate Batch Dimension
    enc_input = tf.expand_dims(inp_sentence, axis= 0) # Axis 0 Corresponds to the Batch

    output = tf.expand_dims([VOCAB_SIZE_FR - 2], axis= 0) # Simulate for Batch

    # Create Several Iteration of Transformer

    for _ in range(MAX_LENGTH):
        predictions = transformer(enc_input, output, False) # If Training is False, Dropout is not Applied. Dimension 1 - for Batch Size. Dimension => (1, seq_length, vocab_size_fr)

        prediction = predictions[:, -1:, :]

        # Get Index of Next word - Higher the probablity, higher the number is next
        predicted_id = tf.cast(tf.argmax(prediction, axis= -1), tf.int32) # Apply along the Last axis
        
        # Test for Ending token
        if predicted_id == VOCAB_SIZE_FR - 1:
            # Translation is Done
            return tf.squeeze(output, axis= 0) # Gets rid of 1st Dimension

        # Add the New Index to the Output
        output = tf.concat([output, predicted_id], axis= -1) # Output is Sequence of Numbers representing the Sentence
    
    return tf.squeeze(output, axis= 0)

In [0]:
# Define a Function Translate that takes input

def translate(sentence):
    output = evaluate(sentence).numpy() # This is an Encoded Version of the Output

    # Decode the Encoded Output - Dont Decode the Starting Token
    predicted_sentence = tokenizer_fr.decode(
        [ i for i in output if i < VOCAB_SIZE_FR - 2] ) # We Kept Only the numbers that corresponds to Real Words

    print("User Input: {}".format(sentence))
    print("Predicted Translation: {}".format(predicted_sentence))

In [0]:
# To Translate

translate("Enter the Sentence")

## Model Sample Tests

In [0]:
# Sample for Transformer Model

# Sample for Creation of Triangular Matrix. This is how we can say that words i(row) will not have access to words j(column). So we want to Mask the words that appears in the Future
print(tf.ones((5, 5)))
print(tf.linalg.band_part(tf.ones((5, 5)), -1, 0))
print(1 - tf.linalg.band_part(tf.ones((5, 5)), -1, 0))

# Sample 2 For Understanding the Masks

def create_padding_mask(seq): # Shape of seq is (batch_size, seq_length)
    # Look for 0's in the Padding token
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32) # Represent it as float, Cast - Casts a tensor to a new datatype.
    return mask[:, tf.newaxis, tf.newaxis, :] # This Mask is applied to Attention Computation & Multihead Attention Sublayer, Before Scaled Dot Product Attention. Add 2 New Dimension
    # Before Applying scaled dotproduct attention, we split data via split proj. So that is the Reason for Adding new Axis. Thanks to Empty Dimension (NewAxis), the Row will be repeated as many times as we need

# Check on for Matrix Multiplication - Create Matrix that cant have access to Future words, So we use Triangular Matrix
def create_look_ahead_mask(seq):
    seq_len = tf.shape(seq)[1] # Get the Shape of Matrix and 1st element
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0) # Create a Triangular Matrix and add 0 to top right part of the Matrix. -1 says we dont want to use this one and 0 says we want j to be >= i
    return look_ahead_mask

print("\n After Defenition, See about Masks and data sample \n")

# Create a Fake Sequence
seq = tf.cast([[837, 835, 0, 273, 9, 0, 0, 0]], tf.int32)
print("Sequence is ", seq, ".See 0's in data are completely masked as 1")

# When we apply Self Attention Mechanism, at beginning of Decoder, we use lookahead mask so we dont have access to Future words, but also the Padding mask

tf.maximum(create_padding_mask(seq), create_look_ahead_mask(seq))
# We get shape as (1, 1, 8, 8), which tells about (batch_size, nb_proj, size_of_matrix, size_of_matrix) - size_of_matrix for Attention weights. 1 says we apply mask(1 corresponds to dont take / 0, see seq matrix)

tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float32)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float32)
tf.Tensor(
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]], shape=(5, 5), dtype=float32)

 After Defenition, See about Masks and data sample 

Sequence is  tf.Tensor([[837 835   0 273   9   0   0   0]], shape=(1, 8), dtype=int32) .See 0's in data are completely masked as 1


<tf.Tensor: shape=(1, 1, 8, 8), dtype=float32, numpy=
array([[[[0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 0., 1., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.]]]], dtype=float32)>

In [0]:
time.sleep(5000)